In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pyproj
import geopandas as gpd
from shapely.geometry import Point
from sklearn.linear_model import Lasso,LassoCV,Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt

In [8]:
for_temple = pd.read_csv("public_quantity_distance.csv")
for_temple['最近金融機構距離']
for_temple['最近廟宇距離']

# concat_data = pd.read_csv("new_concat_data.csv").drop(['最近金融機構距離','最近廟宇距離'], axis=1).reset_index(drop=True)
concat_data = pd.read_csv("new_concat_data.csv").reset_index(drop=True)
origin_concat_data = pd.read_csv("concat_data.csv")
origin_concat_data = origin_concat_data[11751:].drop(['geometry','縱坐標','橫坐標'], axis=1).reset_index(drop=True)
origin_concat_data['最近金融機構距離'] = for_temple['最近金融機構距離']
origin_concat_data['最近廟宇距離'] = for_temple['最近廟宇距離']

new_concat_data = pd.concat([concat_data, origin_concat_data], ignore_index=True)

In [9]:
new_concat_data

,縣市,鄉鎮市區,路名,土地面積,使用分區,移轉層次,總樓層數,主要用途,主要建材,建物型態,...,最近娛樂設施,最近電力資源,最近產業用水及設施,最近工廠,最近商店,最近超級市場,最近大型購物中心,單價,最近金融機構距離,最近廟宇距離
0,台北市,萬華區,水源路,0.100134,None,7,12,住家用,鋼筋混凝土造,住宅大樓(11層含以上有電梯),...,652.635054,97163.858484,91140.483076,136458.703475,169114.219348,5745.826358,44430.815619,1.887258,464.838830,511.010936
1,高雄市,鳳山區,北忠街,0.181921,None,10,15,集合住宅,其他,住宅大樓(11層含以上有電梯),...,1559.977525,5113.742660,1343.918776,2903.279491,155.658006,827.162783,129753.557176,1.489072,282.164695,167.522910
2,新北市,新莊區,福前街,0.085594,None,9,14,集合住宅,鋼筋混凝土造,住宅大樓(11層含以上有電梯),...,2064.189161,98663.458620,95797.250255,140391.552888,171151.625585,7679.115919,50100.306317,2.051217,240.672461,349.220965
3,台北市,萬華區,國興路,0.260510,None,2,12,住家用,鋼筋混凝土造,住宅大樓(11層含以上有電梯),...,236.000039,97989.178309,91572.852428,137028.451622,169899.389221,6628.058791,44431.810363,2.238599,384.139648,62.395726
4,新北市,樹林區,信義街,0.856422,None,2,4,住家用,鋼筋混凝土造,公寓(5樓含以下無電梯),...,414.040551,89883.348426,89058.216769,132618.369151,162396.287236,3630.841437,44903.430177,1.348536,344.240245,303.258173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16290,台南市,中西區,中華西路二段,0.560113,None,5,5,集合住宅,鋼筋混凝土造,公寓(5樓含以下無電梯),...,1430.352664,2299.456395,2453.166311,2327.327780,289.966416,41457.489489,99392.205826,NaN,593.068513,38.967065
16291,台中市,北區,育樂街,-1.020142,None,5,22,集合住宅,鋼筋混凝土造,住宅大樓(11層含以上有電梯),...,695.766012,20389.480616,9611.076087,57213.036785,58227.159368,114549.427458,17848.618959,NaN,224.443765,226.833960
16292,新北市,新店區,安康路二段,-0.662691,None,10,12,住家用,鋼筋混凝土造,住宅大樓(11層含以上有電梯),...,2551.367208,92510.214597,84976.364962,130406.064478,164001.363107,5890.581515,39208.029546,NaN,83.455471,192.899804
16293,台北市,信義區,忠孝東路五段,0.513752,None,2,5,住家用,鋼筋混凝土造,公寓(5樓含以下無電梯),...,354.488321,103641.275468,92628.149190,139392.243969,174862.503993,14040.777133,41940.757037,NaN,133.588326,109.968557


In [10]:
Taipei = new_concat_data[new_concat_data['縣市'] == "台北市"]
Kaohsiung = new_concat_data[new_concat_data['縣市'] == "高雄市"]
New_Taipei = new_concat_data[new_concat_data['縣市'] == "新北市"]
Taoyuan = new_concat_data[new_concat_data['縣市'] == "桃園市"]
Taichung = new_concat_data[new_concat_data['縣市'] == "台中市"]
Tainan = new_concat_data[new_concat_data['縣市'] == "台南市"]
Hsinchu = new_concat_data[(new_concat_data['縣市'] == "新竹縣") | (new_concat_data['縣市'] == "新竹市")]
Else = new_concat_data[(new_concat_data['縣市'] != "台北市") & (new_concat_data['縣市'] != "高雄市") & (new_concat_data['縣市'] != "新北市") & (new_concat_data['縣市'] != "桃園市") & (new_concat_data['縣市'] != "台中市") &( new_concat_data['縣市'] != "台南市") & (new_concat_data['縣市'] != "新竹縣") & (new_concat_data['縣市'] != "新竹市")]
multi = [Taipei, Kaohsiung, New_Taipei, Taoyuan, Taichung, Tainan, Hsinchu, Else]

In [11]:
def ClusterTrainer(city):
    
    # specific = concat_data[concat_data['縣市'] == city]
    specific = city
    
    specific = specific[[
        '縣市', '鄉鎮市區', '路名', '主要用途', '主要建材', '建物型態', '使用分區', '備註', '單價',
        'lng', '最近航空站', '最近幼稚園、托兒所',
        '屋齡', '最近一般公園', '最近碼頭', '最近大型購物中心', 
        '最近超級市場', '最近大學及研究所', '總樓層數',
        '最近國中', '最近生活百貨量販',
        '最近捷運站', '最近電力資源', '最近高鐵站',
        '最近產業用水及設施', '最近飯店', '最近觀光遊憩設施', 
        '最近主要道路建設', 'lat', '最近里鄰公園', '最近娛樂設施', 
        '國中(高)', '國中(中)', '最近火車站', '車位面積', 
        '最近百貨公司', '陽台面積', '移轉層次', '最近停車場地', 
        '最近醫療保健', '最近金融機構', '最近加油站', 
        '土地面積', '主建物面積', '建物面積', '最近工廠', 
        '最近傳統市場', '附屬建物面積', '國中(下)', 
        '最近高中', '最近商店', '最近專科', '最近國小', 
        '最近廟宇距離', '最近金融機構距離'
    ]]
    specific = pd.get_dummies(specific, columns=['鄉鎮市區','路名','主要用途','主要建材','建物型態','使用分區','備註'])
    specific = specific.drop(['縣市'], axis=1)

    # train_test_data = GetSplit(specific)
    Y_train = specific[['單價']].dropna(subset=['單價'])
    X_train = specific.dropna(subset=['單價']).drop(['單價'], axis=1)
    X_test = specific[specific['單價'].isna()].drop(['單價'], axis=1)

    ridge = Ridge()
    parameters = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
    ridge_reg = GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
    result = ridge_reg.fit(X_train.values, Y_train['單價'].values)
    
    return result, X_test

In [12]:
Taipei_res = ClusterTrainer(Taipei)
Kaohsiung_res = ClusterTrainer(Kaohsiung)
New_Taipei_res = ClusterTrainer(New_Taipei)
Taoyuan_res = ClusterTrainer(Taoyuan)
Taichung_res = ClusterTrainer(Taichung)
Tainan_res = ClusterTrainer(Tainan)
Hsinchu_res = ClusterTrainer(Hsinchu)
Else_res = ClusterTrainer(Else)

C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=4.7511e-22): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=8.23886e-22): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.29046e-22): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=4.67125e-22): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarni

In [13]:
dic = {}

In [14]:
for i in range(len(Taipei_res[1])):
    lst = Taipei_res[0].predict(Taipei_res[1].values)
    dic[Taipei_res[1].index[i]] = lst[i]
    
for i in range(len(Kaohsiung_res[1])):
    lst = Kaohsiung_res[0].predict(Kaohsiung_res[1].values)
    dic[Kaohsiung_res[1].index[i]] = lst[i]
    
for i in range(len(New_Taipei_res[1])):
    lst = New_Taipei_res[0].predict(New_Taipei_res[1].values)
    dic[New_Taipei_res[1].index[i]] = lst[i]
    
for i in range(len(Taoyuan_res[1])):
    lst = Taoyuan_res[0].predict(Taoyuan_res[1].values)
    dic[Taoyuan_res[1].index[i]] = lst[i]
    
for i in range(len(Taichung_res[1])):
    lst = Taichung_res[0].predict(Taichung_res[1].values)
    dic[Taichung_res[1].index[i]] = lst[i]
    
for i in range(len(Tainan_res[1])):
    lst = Tainan_res[0].predict(Tainan_res[1].values)
    dic[Tainan_res[1].index[i]] = lst[i]
    
for i in range(len(Hsinchu_res[1])):
    lst = Hsinchu_res[0].predict(Hsinchu_res[1].values)
    dic[Hsinchu_res[1].index[i]] = lst[i]
    
for i in range(len(Else_res[1])):
    lst = Else_res[0].predict(Else_res[1].values)
    dic[Else_res[1].index[i]] = lst[i]

In [15]:
dic = sorted(dic.items(), key=lambda x: x[0])

In [17]:
result = pd.read_csv("public_submission_template.csv")

In [18]:
result['predicted_price'] = dic
result.to_csv("./result.csv", index=False)